In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
url = "https://guide.wisc.edu/undergraduate/business/risk-insurance/business-actuarial-science-bba/#requirementstext"
page = requests.get(url)

In [3]:
text = BeautifulSoup(page.text, "html.parser")

In [4]:
# Get the headings for the previous tables

# First, we find all the tables and only keep the tables containing course info
tables = text.findAll("table", {"class": "sc_courselist" })


In [5]:
def get_ele(x):
    for ele in x:
        return ele.get_text()
    
def replace_that(table):
    a = []
    for row in table:
        r = []
        credits = row.findAll("td", {"class": "hourscol" })
        credits = get_ele(credits)
        r.append(credits)
        
        catg = row.findAll("h2",{"name":"requirementstext"})
        catg = get_ele(catg)
        r.append(catg)
        
        classtags= row.findAll("td", {"class": "codecol"})
        if len(classtags) > 0:
            classtags = get_ele(classtags).replace(u'\xa0',' ')
            classtags = classtags.replace(u'\u200b',' ')
        r.append(classtags)
        
        a.append(r)
    return(a)


In [6]:
#Now, we aim to clean up tb
def clean_up(tb, cat = "None"):
    categories = []
    for row in tb:
        ## Categories
        if row[0] != None and len(row[0]) > 0:
            ## Categories
            if row[0]=="1":
                row[0] = round(int(row[0])/3)
            elif row[0] in ["3","4"]:
                row[0] = round(int(row[0])/3)
            category = ({"Name":cat},
                        row[0],[row[2]])
            categories.append(category)
        elif len(row[2]) > 0:
            category[-1].append(row[2].replace('or ',''))
    return categories

#clean_up(elective_req, "Recommended Electives")

In [17]:
## INFO FROM Major Requirement
major_tbody = tables[1].findAll("tbody")
major_tr=major_tbody[0].findAll("tr")
major_req = replace_that(major_tr)
major = clean_up(major_req, "Actuarial Science Major Requirements")

## INFO FROM ELECTIVE Requirement
elective_tbody = tables[2].findAll("tbody")
elective_tr=elective_tbody[0].findAll("tr")
elective_req = replace_that(elective_tr)
elective = clean_up(elective_req, "Recommended Electives")
elective.pop(-1)
elective.append(({'Name': 'Recommended Electives'}, 3, ['STAT 303', 'STAT 304', 'STAT 305']))
#sp1=elective[-1][-1][0].split('&')[0]
#sp2=elective[-1][-1][0].split('&')[1]
#sp3=elective[-1][-1][0].split('&')[2]
#elective[-1][-1][0]=sp1
#elective[-1][-1].append(sp2)
#elective[-1][-1].append(sp3)
#elective
act_sci_data = major+elective
act_sci_data.pop(-7)
act_sci_data

[({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['MATH/ STAT  431', 'STAT/ MATH  309', 'STAT 311']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['STAT/ MATH  310', 'STAT 312']),
 ({'Name': 'Actuarial Science Major Requirements'}, 0, ['ACT SCI 300']),
 ({'Name': 'Actuarial Science Major Requirements'}, 0, ['ACT SCI 301']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 303']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 650']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 652']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['ACT SCI 651', 'ACT SCI 653']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['ACT SCI 654', 'ACT SCI 655', 'GEN BUS 656']),
 ({'Name': 'Recommended Electives'}, 1, ['MATH 234']),
 ({'Name': 'Recommended Electives'}, 1, ['MATH 340']),
 ({'Name': 'Recommended Electives'}, 1, ['R M I 300']),
 ({'Name': 'Recommended Electives'}, 1, ['FINANCE/ ECON  320']),


In [20]:
for i, row in enumerate(act_sci_data):
    if row[1] == 0:
        act_sci_data[i] = (row[0],1,row[2])
act_sci_data

[({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['MATH/ STAT  431', 'STAT/ MATH  309', 'STAT 311']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['STAT/ MATH  310', 'STAT 312']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 300']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 301']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 303']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 650']),
 ({'Name': 'Actuarial Science Major Requirements'}, 1, ['ACT SCI 652']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['ACT SCI 651', 'ACT SCI 653']),
 ({'Name': 'Actuarial Science Major Requirements'},
  1,
  ['ACT SCI 654', 'ACT SCI 655', 'GEN BUS 656']),
 ({'Name': 'Recommended Electives'}, 1, ['MATH 234']),
 ({'Name': 'Recommended Electives'}, 1, ['MATH 340']),
 ({'Name': 'Recommended Electives'}, 1, ['R M I 300']),
 ({'Name': 'Recommended Electives'}, 1, ['FINANCE/ ECON  320']),
